# Notebook para visualizar el conjunto de datos

Notebook utilizado para hacer representaciones gráficas de lo datos antes de crearlos en streamlit

In [ ]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px

#### Clasificaciones
Representación en un gráfico de líneas los puntos obtenidos durante una temporada sumando los puntos de carreras y carreras formato sprint 

In [ ]:
formatted_results = pd.read_csv('../data/race_and_sprint_results_2000-2024.csv')

# Order by season, round, driverId
formatted_results = formatted_results.sort_values(by=['season', 'round', 'driverId'])

# Calculate accumulated by drivers per season
formatted_results['cumulative_points'] = formatted_results.groupby(['season','driverId'])['points'].cumsum()

# Crear el lineplot
plt.figure(figsize=(14, 8))
sns.lineplot(
    data=formatted_results[formatted_results.season == 2024], 
    x='circuitId',
    y='cumulative_points',
    hue='driverId',
    marker='o',
    palette='tab20'      
)
# Personalizar el gráfico
plt.title('Puntos Ganados por Piloto en Cada Carrera')
plt.xlabel('Carrera')
plt.ylabel('Puntos Ganados')
plt.xticks(rotation=45, ha='right')
plt.legend(title='Piloto', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.ylim(0,425)
plt.yticks(range(0, 425, 25))  # Intervals and jumps


# Display plot
plt.show()

Definición de un diccionario para establecer un color por constructor acorde a los colores que les representa en la realidad

In [ ]:
constructor_color_dict = {
    'Alfa Romeo': '#9C2D2C',      
    'AlphaTauri': '#1E1E1E',     
    'Alpine F1 Team': '#0070BB', 
    'Aston Martin': '#006F42',    
    'Caterham': '#006747',       
    'Ferrari': '#DC0000',         
    'Force India': '#5F4B8B',     
    'Haas F1 Team': '#666666',   
    'HRT': '#F2A900',              
    'Lotus F1': '#1E1E1E',        
    'Lotus': '#F4E300',           
    'Manor Marussia': '#E60012',  
    'Marussia': '#A00000',       
    'McLaren': '#FF5700',         
    'Mercedes': '#00D2BE',        
    'Racing Point': '#F9A9D5',    
    'RB F1 Team': '#1E41FF',      
    'Red Bull': '#1E41FF',        
    'Renault': '#FFCD00',        
    'Sauber': '#003A5C',          
    'Toro Rosso': '#1E41FF',      
    'Virgin': '#E10000',          
    'Williams': '#0046A3'         
}

#### Clasificación de constructores para la misma temporada del gráfico anterior
Consiste en agrupar el conjunto de datos donde tenemos las puntuaciones por el constructor y sumar los puntos obtenidos por los pilotos pertenecientes a cada constructor. Se hace una representación en un gráfico de barras.


In [ ]:
formatted_results_2024 = formatted_results[formatted_results.season == 2024]
columns_for_constructos = ['season', 'constructorId', 'constructorName', 'constructorNationality', 'points', 'weekendPoints']
filtered_constructors_df = formatted_results_2024[columns_for_constructos].copy()

grouped_constructors_df = filtered_constructors_df.groupby(['season','constructorId', 'constructorName'])['weekendPoints'].sum().reset_index()
ordered_constructors_df = grouped_constructors_df.sort_values(by='weekendPoints', ascending=True)

ferrari = ordered_constructors_df[ordered_constructors_df['constructorName'] == 'Sauber']
constructor = ferrari['constructorName'].values[0]
fig_constructors = px.bar(
    ordered_constructors_df, 
    x='weekendPoints', 
    y='constructorName', 
    orientation='h',
    title=f'Clasificación de constructores en la temporada',
    labels={'weekendPoints': 'Points', 'constructorName': 'Constructor'},
    color='constructorName',
    color_discrete_map=constructor_color_dict
)
fig_constructors.show()


#### Victorias

A partir del mismo conjunto de datos, se extraen los pilotos que han quedado en primera posición durante la temporada y se realiza su representación en un gráfico circular

In [ ]:
race_results = formatted_results[formatted_results['season'] == 2024]

wins = race_results[race_results['position'] == 1]
wins.columns

In [ ]:
grouped_wins = wins.groupby(['driverCode', 'driverName', 'driverSurname']).size().reset_index(name='Wins')
grouped_wins = grouped_wins.sort_values(by='Wins', ascending=False)
grouped_wins['driverFullName'] = grouped_wins['driverName'] + " " + grouped_wins['driverSurname']
fig = px.pie(
    grouped_wins,
    names='driverFullName',
    values='Wins',
    title="Distribución de Victorias por Piloto",
    hole=0.2
)

fig.show()

#### Podiums

Sobre el mismo conjunto de datos, se genera un dataframe con las filas cuya posición es inferior a 4, obteniendo así los pilotos que han subido al podium a lo largo de la temporada en los diferentes grandes premios. Se hace una representación en un gráfico circular.

In [ ]:
podiums = race_results[race_results['position'] < 4]

grouped_podiums = podiums.groupby(['driverCode', 'driverName', 'driverSurname']).size().reset_index(name='Podiums')
grouped_podiums = grouped_podiums.sort_values(by='Podiums', ascending=False)
grouped_podiums['driverFullName'] = grouped_podiums['driverName'] + " " + grouped_podiums['driverSurname']
fig = px.pie(
    grouped_podiums,
    names='driverFullName',  
    values='Podiums',  
    title="Distribución de Podiums por Piloto",
    hole=0.2
)
fig.update_layout(
    height=400
)
fig.show()

#### Pitstops

Parte experimental en la que a partir del conjunto de datos del histórico de pitstops se han realizado una serie de representaciones considerando los tiempos.

In [ ]:
pitstops_df = pd.read_csv('../data/cleaned_pitstops.csv')
pitstops_df.head()

In [ ]:
# Function to convert duration to seconds

def duration_to_seconds(duration):
    duration_str = str(duration)
    if ':' in duration_str: 
        minutes, seconds_miliseconds = duration_str.split(':')
        seconds, miliseconds = seconds_miliseconds.split('.')
        return int(minutes) * 60 + int(seconds) + int(miliseconds) / 1000
    else:  
        return float(duration_str)

In [ ]:
pitstops_df['duration'] = pitstops_df['duration'].apply(lambda duration: duration_to_seconds(duration))

In [ ]:
pitstops_df['duration_seconds'] = pitstops_df['duration'].dt.total_seconds()

In [ ]:
# Pitstops duration 2011-2024
# pitstops_df['duration'] = pitstops_df['duration'].astype(float)

fig = px.scatter(pitstops_df[pitstops_df['duration']<50],
                 x='date',
                 y='duration',
                 color='constructorName',
                 color_discrete_map=constructor_color_dict,
                )
fig.show()

##### Boxplots [All seasons]

In [ ]:

fig = px.box(
    pitstops_df[pitstops_df['duration']<50],
    x='date',
    y='duration',
    color='constructorName',
    color_discrete_map=constructor_color_dict,
)
fig.show()

In [ ]:
# Pitstops duration per constructor

fig = px.box(
    pitstops_df[pitstops_df['duration']<50],
    x='constructorName',
    y='duration',
    color='constructorName',
    color_discrete_map=constructor_color_dict,
)
fig.show()

In [ ]:
# Pitstops duration per constructor for 2024
pitstops_2024_df = pitstops_df[pitstops_df['season'] == 2024]

fig = px.box(
    pitstops_2024_df[pitstops_2024_df['duration']<50],
    x='constructorName',
    y='duration',
    color='constructorName',
    color_discrete_map=constructor_color_dict,
)
fig.show()

In [ ]:
# Pit stop duration per circuit
fig = px.box(
    pitstops_df[pitstops_df['duration']<50],
    x='raceName',
    y='duration'
)
fig.show()

##### Histogram [All seasons]

In [ ]:
# Create a histogram
fig = px.histogram(
    pitstops_df[pitstops_df['duration']<50], 
    x="duration"
)
fig.show()